In [1]:
#In this notebook we analyze both the full N=370 RPE and GST datasets,
#computing estimates of the rotation angles, as well as the GST 
#confidence intervals

In [1]:
#Import relevant namespaces.
import pygsti as gst
import numpy as np
import pygsti.construction.std1Q_XY as Std1Q_XY
from pygsti.extras import rpe as RPE
from pygsti.extras.rpe import rpeconstruction as RPEConstr
from pygsti.extras.rpe.rpeconfig_GxPi2_GyPi2_UpDn import rpeconfig_GxPi2_GyPi2_UpDn

Fully specified RPE configuration.


In [3]:
#Declare the particular RPE instance we are interested in
#(X and Y pi/2 rotations)
rpeconfig_inst = rpeconfig_GxPi2_GyPi2_UpDn

In [4]:
#Declare a variety of relevant parameters

gs_target = Std1Q_XY.gs_target
gs_target.set_all_parameterizations('TP')
maxLengths_1024 = [0,1,2,4,8,16,32,64,128,256,512,1024]
fiducials = Std1Q_XY.fiducials
#measFiducials = Std1Q_XY.fiducials
germs = gst.construction.gatestring_list([('Gy',),
 ('Gy','Gy','Gy','Gx',),
 ('Gy','Gx','Gy','Gx','Gx','Gx',),
 ('Gy','Gx','Gy','Gy','Gx','Gx',),
 ('Gy','Gy','Gy','Gx','Gy','Gx',),
 ('Gx',),
 ('Gx','Gy',),
 ('Gx','Gx','Gy','Gx','Gy','Gy',)])
stringListsGST = gst.construction.make_lsgst_lists(gs_target.gates.keys(), fiducials, fiducials, germs, maxLengths_1024)

stringListsRPE = RPEConstr.make_rpe_angle_string_list_dict(10,rpeconfig_inst)

angleList = ['alpha','epsilon','theta']

#NList = [8,16,32,64,128,256]

numKs = len(maxLengths_1024[1:])
numKs_w_0 = len(maxLengths_1024)

numStrsD = {}
numStrsD['RPE'] = [6*i for i in np.arange(1,12)]
numStrsD['GST'] = [len(stringList) for stringList in stringListsGST][1:]

In [5]:
#Load the experimental datasets.
DSGST = gst.io.load_dataset('GST_dataset.txt',cache=True)
DSRPE = gst.io.load_dataset('RPE_dataset.txt',cache=True)

Loading from cache file: GST_dataset.txt.cache
Loading from cache file: RPE_dataset.txt.cache


In [6]:
#Create empty dictionaries and arrays to hold various results.
gsEstD = {}
resultsD = {}
trueD = {}

trials = 1

for method in ['RPE','GST']:
    for angle in angleList:
        resultsArray = np.zeros([numKs,trials,3],float)
        resultsD[(angle,method,'Exp. N=370')] = resultsArray.copy()

In [10]:
#Run GST on full experimental dataset; record final angle estimates.
#This should run in a minute or less.
method = 'GST'

baseKey =  ('Exp. N=370',)

DSTemp = DSGST

resultsGST = gst.do_long_sequence_gst(DSTemp,gs_target,fiducials,fiducials,germs,maxLengths_1024,verbosity=0, gaugeOptParams = {'itemWeights': {'gates':1.0, 'spam':1e-4}})

#resultsGST = Analyze.doMLEAnalysis(DSTemp,gs_target,prepFiducials,measFiducials,germs,maxLengths_1024,constrainToTP=True,advancedOptions={'verbosity':0})
gsEstD[baseKey] = resultsGST.gatesets

alphaKey = ('alpha','GST')+baseKey
epsilonKey = ('epsilon','GST')+baseKey
thetaKey = ('theta','GST')+baseKey

resultsArray = np.zeros([numKs_w_0,1,3],float)

for angle in angleList:
    key = (angle,'GST')+baseKey
    resultsD[key] = resultsArray.copy()

for kInd, k in enumerate(maxLengths_1024):
    gs = gsEstD[baseKey]['iteration estimates'][kInd]#gst.optimize_gauge(gsEstD[baseKey][kInd],'target',targetGateset=gs_target,constrainToTP=True,spamWeight=1e-4)
    resultsD[alphaKey][kInd,0,0] = RPE.extract_alpha(gs,rpeconfig_inst)
    resultsD[epsilonKey][kInd,0,0] = RPE.extract_epsilon(gs,rpeconfig_inst)
    resultsD[thetaKey][kInd,0,0] = RPE.extract_theta(gs,rpeconfig_inst)
    
for angle in angleList:
    trueD[angle] = resultsD[(angle,'GST',)+baseKey][-1,0,0]

DEBUG: optimized gate to min frobenius distance 0.000234144
DEBUG: optimized gate to min frobenius distance 0.000195541
DEBUG: optimized vector to min frobenius distance 0.000107997


In [8]:
#Compute error bars on GST angle estimates.
#This should run in a minute or less.
resultsGST.confidence_level = 95
resultsGSTanglesTable = resultsGST.tables['bestGatesetRotnAxisTable']

    
--- Hessian Projector Optimization for gate CIs (L-BFGS-B) ---
   36s           0.0006786768
   38s           0.0004570397
   41s           0.0002917808
   45s           0.0002752895
   50s           0.0002658809
   52s           0.0002572664
   55s           0.0002209070
   59s           0.0002131450
   62s           0.0002014266
   66s           0.0002007589
  The resulting min sqrt(sum(gateCIs**2)): 0.000200759


In [12]:
#Report GST estimates with 95% CIs.

print "GST full N=370 angle estimates"
for angle, axis in [('alpha','Gx'),('epsilon','Gy')]:
    print angle, "- pi/2 =", trueD[angle] - np.pi/2, '+/-', np.pi*resultsGSTanglesTable[axis]['Angle'][1]

GST full N=370 angle estimates
alpha - pi/2 = 6.39573753674e-05 +/- 4.89225077394e-05
epsilon - pi/2 = 2.73384521914e-05 +/- 3.53597869792e-05


In [13]:
#Encode above GST angle estimates into a "target gateset" to compare RPE results against.
gs_best_truth = RPEConstr.make_paramterized_rpe_gate_set(trueD['alpha'],trueD['epsilon'],trueD['theta'],0,rpeconfig_inst=rpeconfig_inst)

In [14]:
resultsRPE = RPE.analyze_rpe_data(DSRPE,gs_best_truth,stringListsRPE,rpeconfig_inst)
for angle in angleList:
    resultsD[(angle,'RPE','Exp. N=370')][:,0:1,0] = np.array([resultsRPE[angle+'ErrorList']]).T.copy()

In [15]:
#Report RPE estimates along root mean squared error upper bound.
print "RPE full N=370 angle estimates"
for angle in ['alpha','epsilon']:
    print angle, '- pi/2 =', resultsD[(angle,'RPE','Exp. N=370')][-1,0,0]
print "RMSE upper bound = pi/(2*L_max) =", np.pi/(2*1024)

RPE full N=370 angle estimates
alpha - pi/2 = 0.000102359607383
epsilon - pi/2 = 9.93241817457e-05
RMSE upper bound = pi/(2*L_max) = 0.00153398078789


In [16]:
#Report GST estimates with 95% CIs.
for angle in angleList:
    trueD[angle] = resultsD[(angle,'GST','Exp. N=370')][-1,0,0]

print "GST full N=370 angle estimates"
for angle, axis in [('alpha','Gx'),('epsilon','Gy')]:
    print angle, "- pi/2 =", trueD[angle] - np.pi/2, '+/-', np.pi*resultsGSTanglesTable[axis]['Angle'][1]

GST full N=370 angle estimates
alpha - pi/2 = 6.395737537e-05 +/- 4.89225073759e-05
epsilon - pi/2 = 2.73384521923e-05 +/- 3.53597874415e-05
